# Tools

In [22]:
# basic
import requests
import json
import pandas as pd

# gsheet
import pygsheets
# from oauth2client.service_account import ServiceAccountCredentials
import modules.settings as s

# Jupyter notebok setting
pd.options.display.max_columns=None
pd.set_option('max_colwidth', None)  # 設定 value 顯示長度

# 前置作業設定
> **參數**
>- order
    - 2: 日期
    - 14: 符合度
    - 7: 應徵人數
        - 多->少: asc=0
        - 少->多: asc=1
    - 13: 待遇
        - 高->低: asc=0
        - 低->高: asc=1
    - 3: 經歷
        - 多->少: asc=0
        - 少->多: asc=1
>- area
    - 6001001000: 台北市
    - 6001002000: 新北市
>- remoteWork
    - 1: 完全遠端
    - 2: 部分遠端
>- count
    - r.json()['data']['count']
    - 0: 全部
    - 1: 全職
    - 2: 兼職
    - 3: 高階
    - 4: 派遣

## modules

In [9]:
def log_msg(msg):
    print(f">> {msg}")
    
def set_params(keyword, page=1, order=14):
    def set_payload(keyword, page, order):
        payload = {
        #             'ro': '0',
        #             'kwop': '7',
                    'keyword': keyword,
        #             'expansionType': 'area,spec,com,job,wf,wktm',
                    'area': '6001001000,6001002000',  # 台北市、新北市
                    'order': order,
        #             'asc': '0',
                    'page': page,
        #             'mode': 's',
        #             'jobsource': '2018indexpoc',
        #             'langFlag': '0',
        #             'langStatus': '0',
        #             'recommendJob': '1',
    #                 'hotJob': '1',
    #                 'remoteWork':
                 }
    #     if ' ' in payload['keyword']:
    #         payload['keyword'] = payload['keyword'].replace(' ', '%20')

        return payload

    payload = set_payload(keyword, page, order)
    query = f"ro=0&kwop=7&keyword={payload['keyword']}&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&\
            area={payload['area']}&order=14&asc=0&page={payload['page']}&mode=s&jobsource=2018indexpoc&langFlag=0&\
            langStatus=0&recommendJob=1&hotJob=1"

    return query

def get_jobs(keyword, page=1, order=14, pause=False, pause_pageNum=3):
    ### 1. settings ############
    url = 'https://www.104.com.tw/jobs/search/list'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) \
               AppleWebKit/537.36 (KHTML, like Gecko) \
               Chrome/97.0.4692.99 Safari/537.36',
               'Referer': 'https://www.104.com.tw/jobs/search/'
              }
    params = set_params(keyword)
    
    ### 2. 第一次發送 requests.get ############
    # 取得總頁數
    r = requests.get(url=url, headers=headers, params=params)  # page: 1
    log_msg('開始爬取')
    log_msg(f'抓取狀態: {r.status_code}')
    print('搜尋總頁數：', r.json()['data']['totalPage'])
    print('搜尋總職缺筆數：', r.json()['data']['totalCount'])
    
    totalPage = r.json()['data']['totalPage']
    
    # 將第一次爬取到的資料轉為 DataFrame
    df = pd.DataFrame(r.json()['data']['list'])
        
    
    ### 3. 接續爬取後面所有頁數的資料（從 page2 開始）############
    for pageNum in range(2, totalPage+1):
        # 中斷機制（測試用）
        if pause == True:
            if pageNum > pause_pageNum:
                break

        params = set_params(keyword, pageNum)
        r = requests.get(url=url, headers=headers, params=params)
        log_msg(f'開始爬取：第{pageNum}頁')
        
        # 轉換 & 合併 DataFrame
        _df = pd.DataFrame(r.json()['data']['list'])
        df = pd.concat([df, _df])
        log_msg(f'完成爬取：第{pageNum}頁')

    return df

# 開爬

In [10]:
%%time
df = get_jobs('Data Analyst', pause=True)

>> 開始爬取
>> 抓取狀態: 200
搜尋總頁數： 150
搜尋總職缺筆數： 4532
>> 開始爬取第2頁資料
>> 完成第2頁資料爬取
>> 開始爬取第3頁資料
>> 完成第3頁資料爬取
CPU times: user 91.1 ms, sys: 12.1 ms, total: 103 ms
Wall time: 2.43 s


In [13]:
df.shape[0]
# df.jobNo.nunique()
# df

70

# 資料處理

In [14]:
df

,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,optionEdu,period,periodDesc,applyCnt,applyDesc,custNo,custName,coIndustry,coIndustryDesc,salaryLow,salaryHigh,salaryDesc,s10,appearDate,appearDateDesc,optionZone,isApply,applyDate,isSave,descSnippet,tags,landmark,link,jobsource,jobNameRaw,custNameRaw,lon,lat,remoteWorkType,major
0,1,12591928,Personal Banker_財富管理專員_全台,Personal Banker_財富管理專員_全台,1,1,6001001007,台北市信義區,,"工作內容\n財富管理專員必須能獨立作業，並提供Citi Priority客人高品質的服務及多樣化的商品，同時能快速瞭解公司文化及目標，共享資源，合作達成部門目標。財富管理專員也必須能在既有的規章中提出提升整體效率及生產力的新建議、嚴格遵守敏感資訊的傳遞，並能在新的問題中類比過去的類似案例，獨立解決問題。\n\nJob Description\nThe Relationship Associate performs relatively complex assignments. Has direct impact on the business by ensuring the quality of the tasks or services provided. Works under little to no direct supervision. May service an expansive and/or diverse array of products/services. Applies working knowledge of technical and professional principles and concepts and in depth knowledge of team objectives. Understands how assigned duties contribute to the work of the team/unit and how the units coordinate their efforts and resources to achieve the objectives of the function. Ensures the quality and service of self and others. May recommend new options to enhance productivity in accordance with guidelines. Requires tact and diplomacy when exchanging complex or sensitive information with others. Is sensitive to audience diversity. Basic knowledge of the organization, the business and its policies required. Typically responsible for resolving basic to moderately complex problems based on practice and precedence. Able to assess applicability of similar experiences and evaluate options under circumstances not covered by procedures. \n\n工作職責\nResponsibilities:\n\nOn a daily basis, a Personal Banker will be responsible for the following \n1. 積極的以親訪或電話方式開發/維護客戶關係，並提前預測客戶需求、評估客戶風險、開發潛在新客戶\n Actively calling on clients to deepen relationships with affluent clients and proactively owning, responding to and anticipating any future needs, challenges or risks \n2. 發掘客戶潛在需求及期望\n Uncovering customers’ underlying needs and expectations \n3. 提供多元且國際化的金融投資商品，且以完整的資產配置分析，為客戶量身打造專業客製化的財富管理規劃。\n Introducing a wide range of investment, wealth management and treasury products to assist clients in building and strengthening an effective portfolio \n4. 積極提供有關股票、債券、市場條件、投資前途及財務狀況等金融投資商品與國際金融市場之資訊以提升顧客體驗\n Delivering superior customer experience by proactive sharing markets updates, trend and intelligence \n5. 與產品部門合作，為客戶打造客製化解決方案\n Partnering with product specialists and subject matter experts across the bank to struc-ture customized solutions that meet clients’ unique needs \n6. 具有前瞻性，提供差異化的服務為花旗提升競爭力\n Being forward thinking and planning ahead for ways Citi can differentiate itself from com-petitors\n7. 整合知識與資源以實現花旗共同的業務目標\n Leveraging the expertise and capabilities of other Citi businesses to better achieve mutu-al business goals together \n8. 推動創新的解決方案給客戶以進一步發展相關業務。\n Driving innovation on the solutions we provide clients and further developing our business \n9. 經營客戶關係以建立新的客戶聯繫。\n Networking with clients in order to create new business contacts",大學,00,經歷不拘,00011,11~30人應徵,28421312000,花旗(台灣) 商業銀行股份有限公司,,,0000000,0000000,待遇面議,10,20220118,1/18,2101440,0,,0,"工作內容\n財富管理專員必須能獨立作業，並提供Citi Priority客人高品質的服務及多樣化的商品，同時能快速瞭解公司文化及目標，共享資源，合作達成部門目標。財富管理專員也必須能在既有的規章中提出提升整體效率及生產力的新建議、嚴格遵守敏感資訊的傳遞，並能在新的問題中類比過去的類似案例，獨立解決問題。\n\nJob Description\nThe Relationship Associate performs relatively complex assignments. Has direct impact on the business by ensuring the quality of the tasks or services provided. Works under little to no direct supervision. May service an expansive and/or diverse array of products/services. Applies working knowledge of technical and professional principles and concepts and in depth knowledge of team objectives. Understands how assigned duties contribute to the work of the team/unit and how the units coordinate their efforts

# 上傳 google sheet

## gsheet 認證

In [23]:
gc = pygsheets.authorize(service_account_file=s.GSHEETS_PYTHON_CONNECT_PATH)

# Google sheet URL
survey_url = 'https://docs.google.com/spreadsheets/d/1O5FyZ7s9C_8mFXTtO3NrM6ScOwxd742H_GAkIQbCvdQ/edit#gid=0'

sh = gc.open_by_url(survey_url)

## 讀取目標工作表

In [24]:
ws = sh.worksheet_by_title('test')

### 將工作表轉換為 DataFrame
- [set_as_df()](https://pygsheets.readthedocs.io/en/stable/worksheet.html#pygsheets.Worksheet.get_as_df)

In [25]:
dd = ws.get_as_df(start='B2')

In [26]:
dd

,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,optionEdu,period,periodDesc,applyCnt,applyDesc,custNo,custName,coIndustry,coIndustryDesc,salaryLow,salaryHigh,salaryDesc,s10,appearDate,appearDateDesc,optionZone,isApply,applyDate,isSave,descSnippet,tags,landmark,link,jobsource,jobNameRaw,custNameRaw,lon,lat,remoteWorkType,major
0,1,12591928,Personal Banker_財富管理專員_全台,Personal Banker_財富管理專員_全台,1,1,6001001007,台北市信義區,,"工作內容\n財富管理專員必須能獨立作業，並提供Citi Priority客人高品質的服務及多樣化的商品，同時能快速瞭解公司文化及目標，共享資源，合作達成部門目標。財富管理專員也必須能在既有的規章中提出提升整體效率及生產力的新建議、嚴格遵守敏感資訊的傳遞，並能在新的問題中類比過去的類似案例，獨立解決問題。\n\nJob Description\nThe Relationship Associate performs relatively complex assignments. Has direct impact on the business by ensuring the quality of the tasks or services provided. Works under little to no direct supervision. May service an expansive and/or diverse array of products/services. Applies working knowledge of technical and professional principles and concepts and in depth knowledge of team objectives. Understands how assigned duties contribute to the work of the team/unit and how the units coordinate their efforts and resources to achieve the objectives of the function. Ensures the quality and service of self and others. May recommend new options to enhance productivity in accordance with guidelines. Requires tact and diplomacy when exchanging complex or sensitive information with others. Is sensitive to audience diversity. Basic knowledge of the organization, the business and its policies required. Typically responsible for resolving basic to moderately complex problems based on practice and precedence. Able to assess applicability of similar experiences and evaluate options under circumstances not covered by procedures. \n\n工作職責\nResponsibilities:\n\nOn a daily basis, a Personal Banker will be responsible for the following \n1. 積極的以親訪或電話方式開發/維護客戶關係，並提前預測客戶需求、評估客戶風險、開發潛在新客戶\n Actively calling on clients to deepen relationships with affluent clients and proactively owning, responding to and anticipating any future needs, challenges or risks \n2. 發掘客戶潛在需求及期望\n Uncovering customers’ underlying needs and expectations \n3. 提供多元且國際化的金融投資商品，且以完整的資產配置分析，為客戶量身打造專業客製化的財富管理規劃。\n Introducing a wide range of investment, wealth management and treasury products to assist clients in building and strengthening an effective portfolio \n4. 積極提供有關股票、債券、市場條件、投資前途及財務狀況等金融投資商品與國際金融市場之資訊以提升顧客體驗\n Delivering superior customer experience by proactive sharing markets updates, trend and intelligence \n5. 與產品部門合作，為客戶打造客製化解決方案\n Partnering with product specialists and subject matter experts across the bank to struc-ture customized solutions that meet clients’ unique needs \n6. 具有前瞻性，提供差異化的服務為花旗提升競爭力\n Being forward thinking and planning ahead for ways Citi can differentiate itself from com-petitors\n7. 整合知識與資源以實現花旗共同的業務目標\n Leveraging the expertise and capabilities of other Citi businesses to better achieve mutu-al business goals together \n8. 推動創新的解決方案給客戶以進一步發展相關業務。\n Driving innovation on the solutions we provide clients and further developing our business \n9. 經營客戶關係以建立新的客戶聯繫。\n Networking with clients in order to create new business contacts",大學,0,經歷不拘,11,11~30人應徵,28421312000,花旗(台灣) 商業銀行股份有限公司,,,0,0,待遇面議,10,20220118,1/18,2101440,0,,0,"工作內容\n財富管理專員必須能獨立作業，並提供Citi Priority客人高品質的服務及多樣化的商品，同時能快速瞭解公司文化及目標，共享資源，合作達成部門目標。財富管理專員也必須能在既有的規章中提出提升整體效率及生產力的新建議、嚴格遵守敏感資訊的傳遞，並能在新的問題中類比過去的類似案例，獨立解決問題。\n\nJob Description\nThe Relationship Associate performs relatively complex assignments. Has direct impact on the business by ensuring the quality of the tasks or services provided. Works under little to no direct supervision. May service an expansive and/or diverse array of products/services. Applies working knowledge of technical and professional principles and concepts and in depth knowledge of team objectives. Understands how assigned duties contribute to the work of the team/unit and how the units coordinate their efforts and resources t

## 寫入工作表
- [set_dataframe()](https://pygsheets.readthedocs.io/en/stable/worksheet.html#pygsheets.Worksheet.set_dataframe)

In [21]:
ws.set_dataframe(df, start='B2')